# Skills

Making "recipes" happen.

## Notebook setup

Run this cell to set the notebook up. Other sections can be run independently.

In [1]:
%reload_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv
from azure.identity import aio, DefaultAzureCredential, get_bearer_token_provider, AzureCliCredential

from openai import AsyncAzureOpenAI, AzureOpenAI

import logging 
import json
from pathlib import Path

# Set up structured logging to a file.
class JsonFormatter(logging.Formatter):
    def format(self, record) -> str:
        record_dict = record.__dict__
        log_record = {
            'timestamp': self.formatTime(record, self.datefmt),
            'level': record.levelname,
            'session_id': record_dict.get('session_id', None),
            'run_id': record_dict.get('run_id', None),
            'message': record.getMessage(),
            'data': record_dict.get('data', None),
            'module': record.module,
            'funcName': record.funcName,
            'lineNumber': record.lineno,
            'logger': record.name,
        }
        extra_fields = {
            key: value for key, value in record.__dict__.items() 
            if key not in ['levelname', 'msg', 'args', 'exc_info', 'funcName', 'module', 'lineno', 'name', 'message', 'asctime', 'session_id', 'run_id', 'data']
        }
        log_record.update(extra_fields)
        return json.dumps(log_record)

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
modules = ['httpcore.connection', 'httpcore.http11', 'httpcore.sync.connection', 'httpx', 'openai', 'urllib3.connectionpool', 'urllib3.util.retry']
for module in modules:
    logging.getLogger(module).setLevel(logging.ERROR)
if logger.hasHandlers():
    logger.handlers.clear()
data_dir = Path('.data')
if not data_dir.exists():
    data_dir.mkdir()
handler = logging.FileHandler(data_dir / 'logs.jsonl')
handler.setFormatter(JsonFormatter())
logger.addHandler(handler)

# Load the environment variables
load_dotenv()
credential = DefaultAzureCredential()

azure_openai_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT", ""),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION", ""),
    "max_retries": 2,
}

model = azure_openai_config.get("azure_deployment", "gpt-4o")

async_client = AsyncAzureOpenAI(
    **azure_openai_config,
    azure_ad_token_provider=aio.get_bearer_token_provider(
        aio.AzureCliCredential(),
        "https://cognitiveservices.azure.com/.default",
    ),
)

client = AzureOpenAI(
    **azure_openai_config,
    azure_ad_token_provider=get_bearer_token_provider(
        AzureCliCredential(),
        "https://cognitiveservices.azure.com/.default",
    ),
)

## A skill

The Assistants API (and the GPTs that are built with it) need more capabilities than just a few plugins. What we actually want to do is be able to introduce new capabilities not as "tools" but as "skills". A skill is like a tool in that it allows the agent to use various functions for generating chats, but also, we want the agent to be able to run multi-step routines and for skills to be able to interact with one another.

Specifically, a `Skill` provides:

- A nice way to package up a set of capabilities we can provide to an assistant.
- A set of actions that can be made available directly to the assistant (functions) and to the user (commands).
- A set of routines that can be executed by an assistant, potentially in cooperation with the user.

Here is a "Posix" skill that is able to interact with the filesystem in a posix-style way.

In [ ]:
from pathlib import Path
from posix_skill import PosixSkill
from chat_driver import ChatDriverConfig
from context import Context

context = Context("skills-123.posix")

# Define the posix skill.
chat_driver_config = ChatDriverConfig(
    openai_client=async_client,
    model=model,
    instructions="You are an assistant that has access to a sand-boxed Posix shell.",
    context=context,
)

posix_skill = PosixSkill(
    context=context,
    sandbox_dir=Path(".data"),
    chat_driver_config=chat_driver_config,
    mount_dir="/mnt/data",
)

# Registered actions can be called directly from the skill.
await posix_skill.actions.touch("test.txt")
print(await posix_skill.actions.ls(""))

# Note: Look in the .data directory for the output from these actions!

# Chat with the skill.
while True:
    message = input("User: ")
    if message == "":
        break
    print(f"User: {message}", flush=True)
    response = await posix_skill.respond(message)
    print(f"Posix skill: {response.message}", flush=True)

## An assistant

Finally, we have all the building blocks to create a more capable assistant from skills. Our `assistant` library allows you to register skills to an assistant enabling the assistant to perform actions and routines. Here is how simple it is to add our posix skill to an assistant.

In [ ]:
from pathlib import Path
from skill_library import Assistant
from chat_driver import ChatDriverConfig
from posix_skill import PosixSkill


# Define the assistant.
instructions = "You are a helpful assistant."

chat_driver_config = ChatDriverConfig(
    openai_client=async_client,
    model=model,
    instructions=instructions,
)

assistant = Assistant(name="Alice", chat_driver_config=chat_driver_config, session_id="posix-assistant-123")

# Now that the assistant has been created with a context, we can create the
# skills and register them with the assistant's context.

# Define the posix skill. This skill will be used by the assistant. Note that
# some skills may have a conversational interface using a chat driver.
posix_skill = PosixSkill(
    context=assistant.context,
    sandbox_dir=Path(".data"),
    mount_dir="/mnt/data",
    chat_driver_config=ChatDriverConfig(
        openai_client=async_client,
        model=model,
    ),
)

# Register the skill with the assistant.
assistant.register_skills([posix_skill])

# Chat with the assistant.
while True:
    message = input("User: ")
    if message == "":
        break
    print(f"User: {message}", flush=True)
    response = await assistant.generate_response(message)
    print(f"Assistant: {response}", flush=True)

### Running a routine from a skill in an assistant (non-streaming)

In [ ]:
from pathlib import Path
from skill_library import Assistant
from chat_driver import ChatDriverConfig
from posix_skill import PosixSkill

# Define the posix skill. This skill will be used by the assistant. Note that
# some skills may have a conversational interface using a chat driver.


# Define the assistant.
assistant = Assistant(
    name="Alice",
    chat_driver_config=ChatDriverConfig(
        openai_client=async_client,
        model=model,
        instructions="You are a helpful assistant.",
    ),
    session_id="assistant-123",
)

posix_skill = PosixSkill(
    context=assistant.context,
    sandbox_dir=Path(".data"),
    mount_dir="/mnt/data",
    chat_driver_config=ChatDriverConfig(
        openai_client=async_client,
        model=model,
    ),
)

assistant.register_skills([posix_skill])

# Let's see what we can do directly against the assistant.
print(assistant.list_routines())
result = assistant.run_routine("posix.make_home_dir")
print(result)

### An assistant with a message queue

Our assistants also work with the idea of *events* instead of just responses. As the assistant executes actions and routines, a stream of events are generated which can be subscribed to. This makes embedding the assistant in an existing agentic framework easier. For example, we can wrap this assistant in the Semantic Workbench, or in Teams.

In [ ]:
# Apply the nest_asyncio patch to allow asyncio calls handling in this Jupyter
# notebook cell.
import asyncio
import nest_asyncio
from pathlib import Path
from skill_library import Assistant
from chat_driver import ChatDriverConfig
from posix_skill import PosixSkill
nest_asyncio.apply()

# Define the assistant.
chat_driver_config = ChatDriverConfig(
    openai_client=async_client, model=model, instructions="You are a helpful assistant."
)
assistant = Assistant(name="Alice", chat_driver_config=chat_driver_config, session_id="assistant-123")

# Define the posix skill.
posix_skill = PosixSkill(
    context=assistant.context,
    sandbox_dir=Path(".data"),
    mount_dir="/mnt/data",
    chat_driver_config=ChatDriverConfig(
        openai_client=async_client,
        model=model,
    ),
)

# Register the skill with the assistant.
assistant.register_skills([posix_skill])


# Function that allows user input in a non-blocking manner.
async def user_input_handler() -> None:
    while True:
        user_input = await asyncio.to_thread(input, "User: ")
        if user_input == "":
            assistant.stop()
            break
        print(f"User: {user_input}", flush=True)
        await assistant.put_message(user_input)


# Start the user input in a non-blocking way.
input_task = asyncio.create_task(user_input_handler())

# Start the assistant.
async for event in assistant.events:
    print(f"Assistant: {event.message}", flush=True)  # type: ignore

await assistant.wait()
await input_task